<a href="https://colab.research.google.com/github/Zelechos/Pragmatic_IA/blob/master/Pytorch/RedesNeuronales_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch

# Modelos Secuenciales
La forma más sencilla de definir una red neuronal en Pytorch es utilizando la clase Sequentail. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un Perceptrón Multicapa.

In [36]:
Capa_Entrada , Capa_Oculta , Capa_Salida = 784 , 100 , 10
print("Capa Entrada : ",Capa_Entrada)
print("Capa Oculta : ",Capa_Oculta)
print("Capa Salida : ",Capa_Salida)
# Creamos un Perceptron multicapa
model = torch.nn.Sequential(
    torch.nn.Linear(Capa_Entrada , Capa_Oculta),# Conectamos la Capa de entrada con la Capa Oculta
    torch.nn.ReLU(),# usamos la funcion relu para evitar lo valores negartivos ya que no son importantes
    torch.nn.Linear(Capa_Oculta, Capa_Salida)# Conectamos la Capa Oculta con la Capa de Salida
)
# El Perceptron Multicapa(MLP) que acabamos de crear tiene 784 entradas , 100 neuronas en la capa oculta y 10 salidas

Capa Entrada :  784
Capa Oculta :  100
Capa Salida :  10


### A continuacion podemos apreciar un perceptron multicapa mas conocida como una Red Neuronal Artificial RNA
![](https://upload.wikimedia.org/wikipedia/commons/6/64/RedNeuronalArtificial.png)

In [37]:
outputs = model(torch.randn(64, 784))# 64 vectores con 784 valores cada uno es similar a pasarle 64 imagenes de el dataset MNIST

# NOTA IMPORTANTE : casi todos los modeles torch.nn esperan como primera parametro el batch que en este caso le pusimos 64

print(outputs.shape) # Mostramos las longitudes de nuestro tensor

# el tensor que devuelve el model tiene 64 vectores cada uno con 10 valores
#print("Mostramos el Tensor que devuelve el modelo : \n",outputs)

print("notar que cada vector del tensor tiene 10 valores como hemos establecido en la salida del Perceptron Multicapa : \n",
      outputs[0],
      " \n longitud : " ,len(outputs[0]))

torch.Size([64, 10])
notar que cada vector del tensor tiene 10 valores como hemos establecido en la salida del Perceptron Multicapa : 
 tensor([-0.1759, -0.1092, -0.0879, -0.3150, -0.1319, -0.0724,  0.3294, -0.7903,
        -0.1796, -0.1642], grad_fn=<SelectBackward>)  
 longitud :  10


In [38]:
# para entrenar esta Red Neuronal Artificial utilizamos GPU
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

In [39]:
from sklearn.datasets import fetch_openml

# descarga datos

mnist = fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

X.shape, Y.shape

((70000, 784), (70000,))

In [40]:
import numpy as np

# normalización y split

X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.int), Y[60000:].astype(np.int)

print("Mostrando data train  & el data test")
print("Data train : ",X_train , y_train)
print("Data train longitudes : ",len(X_train[0]) , len(y_train))

Mostrando data train  & el data test
Data train :  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [5 0 4 ... 5 6 8]
Data train longitudes :  784 60000


In [41]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [42]:
# convertimos datos a tensores y copiamos en gpu
X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 100
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 1.78028
Epoch 20/100 Loss 1.49424
Epoch 30/100 Loss 1.21947
Epoch 40/100 Loss 1.03661
Epoch 50/100 Loss 0.90101
Epoch 60/100 Loss 0.80729
Epoch 70/100 Loss 0.74094
Epoch 80/100 Loss 0.68945
Epoch 90/100 Loss 0.64276
Epoch 100/100 Loss 0.60408


In [43]:
from sklearn.metrics import accuracy_score
# funcion para evaluar el modelo
def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.93

In [ ]:
# Es una funcion de perdida que viene inplementada en pytorch (funcion de Pérdida de entropía cruzada)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# torch.optim.SGD Implementa descenso de gradiente estocástico (opcionalmente con impulso).
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

In [44]:
model = torch.nn.Sequential(
    torch.nn.Linear(Capa_Entrada , Capa_Oculta),# Conectamos la Capa de entrada con la Capa Oculta
    torch.nn.ReLU(),# usamos la funcion relu para evitar lo valores negartivos ya que no son importantes
    torch.nn.Linear(Capa_Oculta, Capa_Salida)# Conectamos la Capa Oculta con la Capa de Salida
).to("cuda") # para que se cree nuestro Perceptron Multicapa con el GPU

# Creamos la Funcion de Perdida y el Optimizador del descenso del gradiente
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())



Epoch 10/100 Loss 1.74094
Epoch 20/100 Loss 1.38762
Epoch 30/100 Loss 1.12319
Epoch 40/100 Loss 0.96900
Epoch 50/100 Loss 0.85548
Epoch 60/100 Loss 0.76768
Epoch 70/100 Loss 0.70430
Epoch 80/100 Loss 0.65290
Epoch 90/100 Loss 0.61005
Epoch 100/100 Loss 0.57447


0.9315

In [46]:
# creamos una clase que hereda de `torch.nn.Module`

class Model(torch.nn.Module):
    
    # constructor
    def __init__(self, Capa_Entrada, Capa_Oculta, Capa_Salida):
        
        # llamamos al constructor de la clase madre
        super(Model, self).__init__()
        
        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(Capa_Entrada, Capa_Oculta)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(Capa_Oculta, Capa_Salida)
        
    # lógica para calcular las salidas de la red
    def forward(self, Data): #Runtina para ejecutar las funciones que crean cada capa de nuestro Perceptron Multicapa
        Data = self.fc1(Data)
        Data = self.relu(Data)
        Data = self.fc2(Data)
        return Data

In [48]:
# Cramos un modelo utilizando nuestra clase custumizable Model()
model = Model(784, 100, 10)# 784 entradas , 100 neuronas ocultas, 10 salidas 
outputs = model(torch.randn(64, 784))# de batch le ponemos 64 datos que serian 64 imagenes del MNIST tambien de cualquier otro tipo de datset
print(outputs.shape) # retorna un tensor de 64 vectores con 10 elmentos cada uno que corresponden a la salida de mi perceptron multicapa

torch.Size([64, 10])

In [49]:
# cargamos el modelo a la GPU 
model.to("cuda")

# funciones de Perdida & de Optimizacion
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
# funcion para entrenar
model.train()

for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.78164
Epoch 20/100 Loss 1.52910
Epoch 30/100 Loss 1.25986
Epoch 40/100 Loss 1.06017
Epoch 50/100 Loss 0.92408
Epoch 60/100 Loss 0.83111
Epoch 70/100 Loss 0.75902
Epoch 80/100 Loss 0.70582
Epoch 90/100 Loss 0.65758
Epoch 100/100 Loss 0.61766


0.9307

In [ ]:
class Model(torch.nn.Module):
    
    def __init__(self, Capa_Entrada, Capa_Oculta, Capa_Salida):        
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(Capa_Entrada, Capa_Oculta)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(Capa_Oculta, Capa_Salida)
        
        #aqui aplicamos una capa residual 
    def forward(self, x):
        x1 = self.fc1(x)
        x = self.relu(x1)
        x = self.fc2(x + x1)
        return x

In [50]:
# aplica el perceptron multicapa que tiene una capa residual

model = Model(784, 100, 10).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 2.14113
Epoch 20/100 Loss 1.86142
Epoch 30/100 Loss 1.59404
Epoch 40/100 Loss 1.39273
Epoch 50/100 Loss 1.24426
Epoch 60/100 Loss 1.13169
Epoch 70/100 Loss 1.04365
Epoch 80/100 Loss 0.97293
Epoch 90/100 Loss 0.91484
Epoch 100/100 Loss 0.86623


0.8969

In [52]:
# accediendo a las capas del modelo
print(model.fc1)
print(model.relu)
print(model.fc2)


Linear(in_features=784, out_features=100, bias=True)
ReLU()
Linear(in_features=100, out_features=10, bias=True)


In [53]:
# podemos acceder a los pesos 
print(model.fc1.weight)

Parameter containing:
tensor([[-0.0107,  0.0017, -0.0173,  ...,  0.0159, -0.0067, -0.0343],
        [ 0.0179,  0.0131,  0.0221,  ..., -0.0077, -0.0187, -0.0046],
        [-0.0128,  0.0302,  0.0347,  ...,  0.0293, -0.0025,  0.0105],
        ...,
        [ 0.0291, -0.0096,  0.0001,  ..., -0.0259,  0.0149, -0.0143],
        [-0.0096,  0.0013,  0.0156,  ...,  0.0333, -0.0043,  0.0287],
        [ 0.0066, -0.0242, -0.0046,  ...,  0.0029,  0.0051,  0.0195]],
       device='cuda:0', requires_grad=True)


In [54]:
# podemos acceder a los bias
print(model.fc1.bias)

Parameter containing:
tensor([-0.0165,  0.0341, -0.0111,  0.0281, -0.0220,  0.0692,  0.0293, -0.0197,
        -0.0393,  0.0361,  0.0390,  0.0141,  0.0382,  0.0398,  0.0502,  0.0202,
         0.0564,  0.0674,  0.0655, -0.0160,  0.0345, -0.0141,  0.0243,  0.0299,
        -0.0238,  0.0489,  0.0002,  0.0161, -0.0075,  0.0183, -0.0010,  0.0141,
         0.0294,  0.0086,  0.0752,  0.0549, -0.0035,  0.0494,  0.0172, -0.0134,
         0.0210,  0.0549, -0.0488,  0.0236, -0.0253,  0.0144,  0.0057, -0.0122,
         0.0757,  0.0186, -0.0180,  0.0456, -0.0279,  0.0139, -0.0554,  0.0318,
         0.0716,  0.0223,  0.0642,  0.0373, -0.0066,  0.0412,  0.0143, -0.0069,
        -0.0028,  0.0210, -0.0063, -0.0210,  0.0591,  0.0334,  0.0300, -0.0132,
         0.0389,  0.0232,  0.0317,  0.0727, -0.0319,  0.0246,  0.0457,  0.0128,
        -0.0104, -0.0213,  0.0113,  0.0391,  0.0457,  0.0626,  0.0064,  0.0840,
         0.0004,  0.0145,  0.0611, -0.0132,  0.0033,  0.0192,  0.0122,  0.0194,
         0.0421,  

In [59]:
# tambien podemos cambiar el numero de neronas que tiene la capa

model.fc2 = torch.nn.Linear(100,1)
print(model)

Model(
  (fc1): Linear(in_features=1000, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)


In [60]:
# obtener una lista con las capas de una red
list(model.children())

[Linear(in_features=1000, out_features=100, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=1, bias=True)]

In [61]:
# crear nueva red a partir de la lista (excluyendo las útlimas dos capa)
new_model = torch.nn.Sequential(*list(model.children())[:-2])
new_model

Sequential(
  (0): Linear(in_features=1000, out_features=100, bias=True)
)

In [62]:
# crear nueva red a partir de la lista (excluyendo las útlima capa)
new_model = torch.nn.ModuleList(list(model.children())[:-1])
new_model

ModuleList(
  (0): Linear(in_features=1000, out_features=100, bias=True)
  (1): ReLU()
)